# Elementary matrices, Gaussian elimination, reduced echelon form and row equivalence



In [1]:
include("linalg/rowequiv.jl")

using .RowEquivalence

## Elementary matrices

TODO

## Gaussian elimination

TODO: motivation, connection to elementary matrices

In [2]:
mat = [1  3  1  9;
       1  1 -1  1;
       3 11  5 35]

3×4 Matrix{Int64}:
 1   3   1   9
 1   1  -1   1
 3  11   5  35

In [3]:
reducedechelon_steps(mat)

LoadError: InexactError: Int64(-0.5)

TODO: explain that every element in M must be invertible for Gaussian elimination, hence must for example convert to real or rational values.

In [4]:
mat = convert(Matrix{Float64}, mat)

3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 1.0   1.0  -1.0   1.0
 3.0  11.0   5.0  35.0

In [5]:
steps, ops = reducedechelon_steps(mat, return_operations=true)
length(ops)

5

In [6]:
function prettyprint(x)
    show(stdout, "text/plain", x)
    println()
end

prettyprint (generic function with 1 method)

In [7]:
prevstep = mat
for (i, (step, op)) in enumerate(zip(steps, ops))
    println("-------")
    println("step ", i, ":")
    println("-------")
    println("apply")
    prettyprint(op)
    println("to")
    prettyprint(prevstep)
    println("=>")
    prettyprint(step)
    println()
    prevstep = step
end;

-------
step 1:
-------
apply
3×3 Matrix{Float64}:
  1.0  0.0  0.0
 -1.0  1.0  0.0
  0.0  0.0  1.0
to
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 1.0   1.0  -1.0   1.0
 3.0  11.0   5.0  35.0
=>
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 0.0  -2.0  -2.0  -8.0
 3.0  11.0   5.0  35.0

-------
step 2:
-------
apply
3×3 Matrix{Float64}:
  1.0  0.0  0.0
  0.0  1.0  0.0
 -3.0  0.0  1.0
to
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 0.0  -2.0  -2.0  -8.0
 3.0  11.0   5.0  35.0
=>
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 0.0  -2.0  -2.0  -8.0
 0.0   2.0   2.0   8.0

-------
step 3:
-------
apply
3×3 Matrix{Float64}:
  1.0   0.0   0.0
 -0.0  -0.5  -0.0
  0.0   0.0   1.0
to
3×4 Matrix{Float64}:
 1.0   3.0   1.0   9.0
 0.0  -2.0  -2.0  -8.0
 0.0   2.0   2.0   8.0
=>
3×4 Matrix{Float64}:
 1.0  3.0  1.0  9.0
 0.0  1.0  1.0  4.0
 0.0  2.0  2.0  8.0

-------
step 4:
-------
apply
3×3 Matrix{Float64}:
 1.0  -3.0  0.0
 0.0   1.0  0.0
 0.0   0.0  1.0
to
3×4 Matrix{Float64}:
 1.0  3.0  1.0  9.0
 0

## Reduced echelon form

TODO

In [8]:
convert(Matrix{Int}, steps[end])

3×4 Matrix{Int64}:
 1  0  -2  -3
 0  1   1   4
 0  0   0   0

In [9]:
reducedechelon(mat)

3×4 Matrix{Float64}:
 1.0  0.0  -2.0  -3.0
 0.0  1.0   1.0   4.0
 0.0  0.0   0.0   0.0

## Row equivalence

TODO

## Matrix rank

TODO

## Matrix inverse

TODO

note that there are more efficient algorithms